#### 1. 필요한 라이브러리 및 패키지 임포트

In [6]:
import pandas as pd # 데이터 조작과 분석 시 필요
import re # 정규 표현식을 지원하여 패턴 매칭과 텍스트 조작 시 필요
import time # 시간 측정 시 필요
import random # 난수 생성 시 필요
import requests # HTTP요청을 만들고 웹 서비스와 상호작용 시 필요
from bs4 import BeautifulSoup as bs # HTML이나 XML 문서를 구문 분석하고 탐색하기 위해 필요
from selenium import webdriver # 브라우저 자동화를 위해 필요
from selenium.common.exceptions import UnexpectedAlertPresentException #예기치 않은 경고나 팝업 발생 시 예외 처리에 필요
import os
import sys
import urllib.request
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

#### 2. 네이버 API로 블로그 url 추출(데이터 수집 개수 제한 있음) 

In [41]:
blog_url_list=[]

# 네이버 developers에서 내 정보 확인
client_id = "INPUT_client ID"
client_secret = "INPUT_client Secret"

query = urllib.parse.quote("INPUT_검색키워드")

#100개 이상 수집 시 오류 발생
idx=0
display=10 # 한번에 가져오는 페이지 개수 설정
start=1
end=100


for idx in range(start,end,display):
    url = "https://openapi.naver.com/v1/search/blog?query=" + query + "&display=" + str(display) + "&start=" + str(idx) # 네이버 블로그 검색 API 호출 URL 생성성
    
    # D요청 객체 생성 및 헤더 추가
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)

    # API 요청 및 응답 확인
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode==200): # 정상 반응
        response_body = response.read()
        data=response_body.decode('utf-8')
        data = json.loads(data)
        links = [item['link'] for item in data['items']]
        for link in links:
            blog_url_list.append(link) # 블로그 포스트 url 저장
        
    else: # 예외처리
        print("Error Code:" + rescode)

#### 3. 크롤링으로 블로그 url 추출 

##### 3.1 검색 페이지 url 생성 함수

In [ ]:
def makePgNum(num): # 페이지 넘버링 함수
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+29*(num-1)


# 크롤링할 url 생성 함수
def makeUrl(search,start_pg,end_pg):
    if start_pg == end_pg:
        page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query=" + search + "&start=" + str(page)
        print("생성url: ",url)
        return url
    
    else:
        urls= []
        for i in range(start_pg,end_pg+1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ",urls)
        return urls

##### 3.2 검색 페이지 url 생성

In [ ]:
# 검색하고자 하는 키워드를 입력
search = input("검색할 키워드를 입력해주세요 :")

# 네이버 블로그는 한 페이지 당 30개의 블로그 정보 확인 가능함 (예를 들어, 1 - 10페이지 입력 시 300개의 정보 크롤링 가능)
# 검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요 :"))
# 검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요 :"))

# naver url 생성
search_urls = makeUrl(search,page,page2)

##### 3.3 블로그 포스트 url 추출

In [ ]:
# 블로그 포스트 url 저장 리스트
blog_url_list=[]

for i in search_urls:
    r=requests.get(i) # requests.get(url)로 요청
    time.sleep(random.radint(1,3)) # 요청시간 랜덤 조정으로 크롤링 차단 방지

    soup=bs(r.text, "html.parser") # html.parser로 파싱
    
    # 제목, 본문 텍스트 영역 추출
    title_areas=soup.find_all("div", {'class':'title_area'})
    user_boxes = soup.find_all("div", {'class':'user_box'})
    
    for url,pro in zip(title_areas, user_boxes):
        if '광고' in pro.text: # 포스트 옆에 광고 표시가 떠있으면 제외
            pass
        
        else:
            url_text = str(url)
            pattern = r'href="([^"]+)"'
            links = re.findall(pattern, url_text) # 정규식으로 링크 추출
            blog_url_list.append(links)

print(len(blog_url_list),' 개 블로그 url주소 수집') # 수집된 포스트 url 개수 출력

#### 4. url 저장

In [ ]:
# 수집 후 blog_url_list에 중복된 url이 있는지 확인 후 중복 제거 (순서 보존)
filtered_urls = [url for url in blog_url_list if 'blog.naver' in url[0]]
unique_urls = list(dict.fromkeys(url[0] for url in filtered_urls))

# 최종 블로그 url 목록 개수 출력 
print(len(unique_urls))

In [48]:
# url 파일 저장
url = pd.DataFrame(unique_urls)
url.to_csv('naverblog_url.csv',encoding='utf-8-sig')

-------------

#### 5. 블로그 포스트 본문 크롤링

In [49]:
# 저장한 url 파일 불러오기
url = pd.read_csv('naverblog_url.csv') #url 저장위치, 파일명 입력
blog_url_list = list(url['0']) # url 파일의 첫번째(0) 컬럼을 blog_url_list로 저장
len(blog_url_list) # 개수 확인

1000

In [50]:
no_naver_blog=[] # 네이버 블로그 외 다른 주소 필터링

start = time.time() # 소요 시간 측정

# 수집 항목
time_list = [] # 작성 시간
review_list = [] # 본문
url_list = [] # url 주소
count = 0 # 진행 상황을 파악하기 위한 크롤링 개수 카운팅

driver = webdriver.Chrome() 

for url in blog_url_list: 
    if 'naver' in url: 

        count += 1
        driver.get(url)
        time.sleep(random.radint(1,3)) # 요청시간 랜덤 조정으로 크롤링 차단 방지

        try: # 네이버 블로그 크롤링
            driver.switch_to.frame('mainFrame') # 네이버는 여러 mainFrame으로 구성되어 있어서, 원하는 프레임으로 전환
            time.sleep(0.7) 
            soup = bs(driver.page_source, 'lxml') # 현재 페이지의 HTML 소스 파싱
            
            postview = soup.find('div', attrs={'id': re.compile('post-view.+')}).get_text() # 게시글 요소 텍스트 추출
            reg = re.compile(r'[\s+]') # 공백 처리 패턴 정의 (\s 공백, + 1개 이상 연속) 
            postview_reg = reg.sub(' ',postview) # 한 칸 공백으로 대체
            
            try:
                # 게시물 작성 시간
                timeline = driver.find_element("xpath", "//span[@class='se_publishDate pcol2']") # 시간 요소 저장(유형 1)
            except:
                timeline = driver.find_element("xpath", "//p[@class='date fil5 pcol2 _postAddDate']") # 시간 요소 저장(유형2)
            timeline = timeline.text # 저장
             
            # 수집한 내용 각 리스트 추가
            time_list.append(timeline)
            review_list.append(postview_reg)
            url_list.append(url)
            time.sleep(random.uniform(0.5,1.3))

        except UnexpectedAlertPresentException: # 예외처리
            print('Error extracting data')
            pass
    else: # 제외 링크 확인용
        no_naver_blog.append(url)
        print('네이버 블로그 아닌 url: ',url)
        
    if count%10 ==0: # 진행 상황 확인용
        print('{}개 블로그 크롤링 완료'.format(count)) 

10개 블로그 크롤링 완료
20개 블로그 크롤링 완료
30개 블로그 크롤링 완료
40개 블로그 크롤링 완료
50개 블로그 크롤링 완료
60개 블로그 크롤링 완료
70개 블로그 크롤링 완료
80개 블로그 크롤링 완료
90개 블로그 크롤링 완료
100개 블로그 크롤링 완료
110개 블로그 크롤링 완료
120개 블로그 크롤링 완료
130개 블로그 크롤링 완료
140개 블로그 크롤링 완료
150개 블로그 크롤링 완료
160개 블로그 크롤링 완료
170개 블로그 크롤링 완료
180개 블로그 크롤링 완료
190개 블로그 크롤링 완료
200개 블로그 크롤링 완료
210개 블로그 크롤링 완료
220개 블로그 크롤링 완료
230개 블로그 크롤링 완료
240개 블로그 크롤링 완료
250개 블로그 크롤링 완료
260개 블로그 크롤링 완료
270개 블로그 크롤링 완료
280개 블로그 크롤링 완료
290개 블로그 크롤링 완료
300개 블로그 크롤링 완료
310개 블로그 크롤링 완료
320개 블로그 크롤링 완료
330개 블로그 크롤링 완료
340개 블로그 크롤링 완료
350개 블로그 크롤링 완료
360개 블로그 크롤링 완료
370개 블로그 크롤링 완료
380개 블로그 크롤링 완료
390개 블로그 크롤링 완료
400개 블로그 크롤링 완료
410개 블로그 크롤링 완료
420개 블로그 크롤링 완료
430개 블로그 크롤링 완료
440개 블로그 크롤링 완료
450개 블로그 크롤링 완료
460개 블로그 크롤링 완료
470개 블로그 크롤링 완료
480개 블로그 크롤링 완료
490개 블로그 크롤링 완료
500개 블로그 크롤링 완료
510개 블로그 크롤링 완료
520개 블로그 크롤링 완료
530개 블로그 크롤링 완료
540개 블로그 크롤링 완료
550개 블로그 크롤링 완료
560개 블로그 크롤링 완료
570개 블로그 크롤링 완료
580개 블로그 크롤링 완료
590개 블로그 크롤링 완료
600개 블로그 크롤링 완료
610개 블로그 크롤링 완료
620개 블로그 크롤링 완료
630개 블로그 크롤링 완료
6

#### 6. 데이터 저장 

In [ ]:
data = {'time' : time_list, # 시간
        'review' : review_list, # 게시글
        'url': url_list} # url
df = pd.DataFrame(data)
df = df.drop_duplicates() # 중복제거

print('크롤링 시간: ', time.time()-start)
print('크롤링 개수:', len(review_list))
df.head()

In [52]:
df.to_csv('naverblog_result.csv', encoding = "utf-8-sig")